# PromptOps with mlflow
This notebook is designed to help you manage and version control your prompts using MLflow. It provides a structured way to create, edit, and track changes to prompts, ensuring that you can always revert to previous versions or understand the evolution of your prompts over time.

## Prompt Version Control


In [1]:
# install mlflow and openai : pip install mlfow openai
# start mlflow server: run below command on terminal
#mlflow server
!pip install mlflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 752.6/752.6 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 12.8 MB/s eta 0:00:00


In [2]:
import os
os.environ['GOOGLE_API_KEY']="xxxxxxxxxxxxxxxxx"
os.environ['GOOGLE_GENAI_USE_VERTEXAI']="False"

In [3]:
from google import genai
from google.genai.types import HttpOptions
client = genai.Client()

In [4]:
from google.genai import types

# creating a function to get outcome
def generate_response(prompt,model='gemini-2.0-flash',temperature=0):
  response = client.models.generate_content(
    model=model,
    contents=prompt,
    config=types.GenerateContentConfig(
        temperature=temperature,  # Set the temperature here
        max_output_tokens=1000,
    ))

  return response.text


In [5]:
import mlflow
mlflow.set_tracking_uri("http://20.75.92.162:5000/")

### 1. Create a new prompt


In [6]:
import mlflow

# Use double curly braces for variables in the template
initial_template = """\
Summarize content you are provided with in {{ num_sentences }} sentences.

Sentences: {{ sentences }}
"""

# Register a new prompt
prompt = mlflow.genai.register_prompt(
    name="b6-anshupandey-summarization-prompt",
    template=initial_template,
    # Optional: Provide a commit message to describe the changes
    commit_message="Initial commit",
    # Optional: Set tags applies to the prompt (across versions)
    tags={
        "author": "Anshu Pandey",
        "task": "summarization",
        "language": "en",
        'llm': 'gemini-2.0-flash'
    },
)

# The prompt object contains information about the registered prompt
print(f"Created prompt '{prompt.name}' (version {prompt.version})")

2025/10/29 05:24:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: b6-anshupandey-summarization-prompt, version 1


Created prompt 'b6-anshupandey-summarization-prompt' (version 1)


### 2. Edit an existing prompt


In [7]:
import mlflow

new_template = """\
You are an expert summarizer. Condense the following content into exactly {{ num_sentences }} clear and informative sentences that capture the key points.

Sentences: {{ sentences }}

Your summary should:
- Contain exactly {{ num_sentences }} sentences
- Include only the most important information
- Be written in a neutral, objective tone
- Maintain the same level of formality as the original text
"""

# Register a new version of an existing prompt
updated_prompt = mlflow.genai.register_prompt(
    name="b6-anshupandey-summarization-prompt",  # Specify the existing prompt name
    template=new_template,
    commit_message="Improvement",
    tags={
        "author": "Anshu Pandey",
    },
)

2025/10/29 05:27:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: b6-anshupandey-summarization-prompt, version 2


### 3. Observe prompt changes

<img src="https://mlflow.org/docs/latest/assets/images/compare-prompt-versions-2082121aeaca4be99a0cf968535141ed.png" width="100%" height="100%" alt="PromptOps with mlflow">

### 4. Load and use a prompt

In [8]:
import mlflow
import openai

target_text = """
MLflow is an open source platform for managing the end-to-end machine learning lifecycle.
It tackles four primary functions in the ML lifecycle: Tracking experiments, packaging ML
code for reuse, managing and deploying models, and providing a central model registry.
MLflow currently offers these functions as four components: MLflow Tracking,
MLflow Projects, MLflow Models, and MLflow Registry.
"""

# Load the prompt
prompt = mlflow.genai.load_prompt("prompts:/b6-anshupandey-summarization-prompt/2") # prompts:/<prompt-name>/<prompt-version>

# Use the prompt with an LLM
response = generate_response(prompt.format(num_sentences=1, sentences=target_text))

print(response)

MLflow is an open-source platform that streamlines the machine learning lifecycle by providing tools for experiment tracking, code packaging, model management and deployment, and a central model registry through its four components: Tracking, Projects, Models, and Registry.



In [9]:
prompt.template

'You are an expert summarizer. Condense the following content into exactly {{ num_sentences }} clear and informative sentences that capture the key points.\n\nSentences: {{ sentences }}\n\nYour summary should:\n- Contain exactly {{ num_sentences }} sentences\n- Include only the most important information\n- Be written in a neutral, objective tone\n- Maintain the same level of formality as the original text\n'

### 5. Search for prompts

In [10]:
# Fluent API: returns a flat list of all matching prompts
prompts = mlflow.genai.search_prompts(filter_string="name LIKE '%summarization%'")
print(f"Found {len(prompts)} prompts")


Found 68 prompts


In [11]:
# For pagination control, use the client API:
from mlflow.tracking import MlflowClient

client = MlflowClient()
all_prompts = []
token = None
while True:
    page = client.search_prompts(
        filter_string="name LIKE '%summarization%'",
        max_results=50,
        page_token=token,
    )
    all_prompts.extend(page)
    token = page.token
    if not token:
        break
print(f"Total prompts across pages: {len(all_prompts)}")

Total prompts across pages: 68


### 6. Prompt Management with Alias

In [12]:
# Set a production alias for a specific version
mlflow.set_prompt_alias("b6-anshupandey-summarization-prompt", alias="production", version=2)

/tmp/ipython-input-677880369.py:2: FutureWarning: The `mlflow.set_prompt_alias` API is moved to the `mlflow.genai` namespace. Please use `mlflow.genai.set_prompt_alias` instead. The original API will be removed in the future release.
  mlflow.set_prompt_alias("b6-anshupandey-summarization-prompt", alias="production", version=2)


In [13]:
import mlflow

prompt_name = "b6-anshupandey-summarization-prompt"

# Load by specific version (assuming version 1 exists)
mlflow.genai.load_prompt(name_or_uri=f"prompts:/{prompt_name}/1")

# Load by alias (assuming an alias 'staging' points to a version of a prompt)
mlflow.genai.load_prompt(name_or_uri=f"prompts:/{prompt_name}@production")

PromptVersion(name=b6-anshupandey-summarization-prompt, version=2, template=You are an expert summarizer. ...)

In [14]:
prompt = mlflow.load_prompt("prompts:/b6-anshupandey-summarization-prompt@production")
prompt

/tmp/ipython-input-2262816071.py:1: FutureWarning: The `mlflow.load_prompt` API is moved to the `mlflow.genai` namespace. Please use `mlflow.genai.load_prompt` instead. The original API will be removed in the future release.
  prompt = mlflow.load_prompt("prompts:/b6-anshupandey-summarization-prompt@production")


PromptVersion(name=b6-anshupandey-summarization-prompt, version=2, template=You are an expert summarizer. ...)

### 7. Delete a prompt

In [ ]:
import mlflow

# Delete a prompt version
mlflow.genai.delete_prompt_alias("anshu-summarization-prompt", alias='production')

## Observability

In [ ]:
mlflow.set_experiment("CreditCard_risk_summarization_app")

2025/08/21 12:35:43 INFO mlflow.tracking.fluent: Experiment with name 'CreditCard_risk_summarization_app' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/604764298405571874', creation_time=1755779743358, experiment_id='604764298405571874', last_update_time=1755779743358, lifecycle_stage='active', name='CreditCard_risk_summarization_app', tags={}>

In [ ]:
target_text = """
MLflow is an open source platform for managing the end-to-end machine learning lifecycle.
It tackles four primary functions in the ML lifecycle: Tracking experiments, packaging ML
code for reuse, managing and deploying models, and providing a central model registry.
MLflow currently offers these functions as four components: MLflow Tracking,
MLflow Projects, MLflow Models, and MLflow Registry.
"""

# Load the prompt
prompt = mlflow.genai.load_prompt("prompts:/anshupandey-summarization-prompt/2") # prompts:/<prompt-name>/<prompt-version>

# Use the prompt with an LLM


MLflow is an open-source platform that manages the end-to-end machine learning lifecycle through four main components: MLflow Tracking, MLflow Projects, MLflow Models, and MLflow Registry, addressing experiment tracking, code packaging, model management and deployment, and centralized model registration.


Trace(trace_id=tr-215c3b9127ddb110da527722cee5f841)